In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from scipy.stats import chi2_contingency

df = pd.read_csv ('/Users/pao.tovar/Downloads/atusrost_2019.csv')
df2 = pd.read_csv ('/Users/pao.tovar/Downloads/atusresp_2019.csv')
df3 = pd.read_csv ('/Users/pao.tovar/Downloads/atuscps_2019.csv')
df3.head()

,TUCASEID,TULINENO,GEREG,GEDIV,GESTFIPS,GTMETSTA,GTCBSA,GTCO,HEFAMINC,HEHOUSUT,...,PXPDEMP1,PXNMEMP1,PXPDEMP2,PXNMEMP2,PECERT1,PECERT2,PECERT3,PXCERT1,PXCERT2,PXCERT3
0,20190101190022,1,3,7,48,1,26420,0,6,1,...,0,0,0,0,2,-1,-1,20,0,0
1,20190101190031,1,2,3,26,1,24340,0,14,6,...,1,1,-1,-1,2,-1,-1,0,0,0
2,20190101190031,2,2,3,26,1,24340,0,14,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,20190101190031,3,2,3,26,1,24340,0,14,6,...,1,1,-1,-1,2,-1,-1,0,0,0
4,20190101190031,4,2,3,26,1,24340,0,14,6,...,1,1,-1,-1,2,-1,-1,42,0,0


In [2]:
sex = df3['PESEX'].dropna()

In [3]:
male = df3['PESEX'] == 1
female = df3['PESEX'] == 2

In [4]:
def function1(row):
    if row['PESEX'] == 1:
        return 'male'
    elif row['PESEX'] == 2:
        return 'female'

In [5]:
df3['PESEX'] = df3.apply(function1, axis=1)

In [6]:
status= df3['PENLFACT'].dropna()

In [7]:
def function2(row):
    if row['PENLFACT'] == 1:
        return 'Disability'
    elif row['PENLFACT'] == 2:
        return 'Ill'
    elif row['PENLFACT'] == 3:
        return 'School'
    elif row['PENLFACT'] == 4:
        return 'Taking care of h or f'
    elif row['PENLFACT'] == 5:
        return 'In retirement'
    elif row['PENLFACT'] == 6:
        return 'Something else/other'
    else:
        return 'NaN'

In [8]:
df3['PENLFACT'] = df3.apply(function2, axis=1)

In [9]:
table1 = pd.crosstab(index=df3["PENLFACT"], columns=df3["PESEX"])

In [54]:
table1

PESEX,female,male,Total,female_per
PENLFACT,,,,
Taking care of h or f,2760,305,3065,0.900489
Disability,76,75,151,0.503311
Ill,53,44,97,0.546392
In retirement,18,19,37,0.486486
School,1944,2105,4049,0.480119
Something else/other,180,272,452,0.398230


In [10]:
table1 = table1.reindex(['Taking care of h or f','Disability', 'Ill', 'In retirement', 'School', 'Something else/other'])
table1

PESEX,female,male
PENLFACT,,
Taking care of h or f,2760,305
Disability,76,75
Ill,53,44
In retirement,18,19
School,1944,2105
Something else/other,180,272


In [11]:
table1.columns

Index(['female', 'male'], dtype='object', name='PESEX')

In [12]:
total1 = 0
for gender in table1.columns:
    total1 += np.sum(table1[gender])
print(total1)

7851


In [13]:
chi2, p, dof, expected = stats.chi2_contingency(table1)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)

chi2: 1487.7285581524884
p: 0.0
dof: 5
expected: [[1964.08291937 1100.91708063]
 [  96.76232327   54.23767673]
 [  62.15857853   34.84142147]
 [  23.70997325   13.29002675]
 [2594.64004585 1454.35995415]
 [ 289.64615972  162.35384028]]


In [14]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(table1)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)

chi2: 1487.7285581524884
p: 0.0
dof: 5
expected: [[1964.08291937 1100.91708063]
 [  96.76232327   54.23767673]
 [  62.15857853   34.84142147]
 [  23.70997325   13.29002675]
 [2594.64004585 1454.35995415]
 [ 289.64615972  162.35384028]]


Statistical analysis from chi-squared test from independence indicated that there was an association between gender and status
(p < 0.00).
 chi^2(5, N = 7851)=1487.73; p < 0.001

In [15]:
table1['Total'] = table1['male'] + table1['female']
table1['female_per'] = table1['female'] / table1['Total']
table1

PESEX,female,male,Total,female_per
PENLFACT,,,,
Taking care of h or f,2760,305,3065,0.900489
Disability,76,75,151,0.503311
Ill,53,44,97,0.546392
In retirement,18,19,37,0.486486
School,1944,2105,4049,0.480119
Something else/other,180,272,452,0.398230


In [16]:
def CI_prop_diff(s1, s2, n1, n2):
    p1 = s1 / n1
    p2 = s2 / n2
    se2p1 = p1 * (1 - p1) / n1
    se2p2 = p2 * (1 - p2) / n2
    se2 = se2p1 + se2p2
    se = np.sqrt(se2)
    low = (p1 - p2) - 1.96 * se
    up = (p1 - p2) + 1.96 * se
    print("Proportion difference is:", p1 - p2)
    print("95% CI is: ({}, {})".format(low, up))

In [17]:
CI_prop_diff(s1=2760, s2=305, n1=3065, n2=4786)

Proportion difference is: 0.8367618577566842
95% CI is: (0.8241046248947469, 0.8494190906186215)


My hypothesis is that there is an association between gender and status, concretely, taking care of house or children.
The data provide a strong evidence that the probability of women taking care or the house of family is bigger than
the probability of men doing so.  chi^2(5, N = 7851)=1487.73; p  < 0.001. 
The observed proportion difference was 0.837 (95% CI = [0.824,0.849]).

In [18]:
race= df3['PTDTRACE'].dropna()

In [19]:
def function3(row):
    if row['PTDTRACE'] == 1:
        return 'White_only'
    elif row['PTDTRACE'] == 2:
        return 'Black_only'
    elif row['PTDTRACE'] == 4:
        return 'Asian_only'
    elif row['PTDTRACE'] == 3 or row['PTDTRACE'] == 5 or row['PTDTRACE'] == 6 or row['PTDTRACE'] ==7 or row['PTDTRACE'] ==8 or row['PTDTRACE'] ==8 or row['PTDTRACE'] ==9 or row['PTDTRACE'] ==10 or row['PTDTRACE'] == 11 or row['PTDTRACE'] ==12 or row['PTDTRACE'] ==13 or row['PTDTRACE'] ==14or row['PTDTRACE'] ==15 or row['PTDTRACE'] ==16 or row['PTDTRACE'] ==17 or row['PTDTRACE'] ==18 or row['PTDTRACE'] ==19 or row['PTDTRACE'] ==20 or row['PTDTRACE'] ==21 or row['PTDTRACE'] ==22 or row['PTDTRACE'] ==23 or row['PTDTRACE'] ==24 or row['PTDTRACE'] ==25 or row['PTDTRACE'] ==26:
        return 'Other'
    else:
        return 'NaN'

In [20]:
df3['PTDTRACE'] = df3.apply(function3, axis=1)

In [21]:
table2 = pd.crosstab(index=df3['PENLFACT'], columns=df3["PTDTRACE"])

In [22]:
table2

PTDTRACE,Asian_only,Black_only,Other,White_only
PENLFACT,,,,
Disability,7,39,5,100
Ill,6,14,5,72
In retirement,5,4,3,25
NaN,2713,7858,2073,43790
School,285,591,189,2984
Something else/other,27,111,19,295
Taking care of h or f,258,320,98,2389


In [23]:
table2.index

Index(['Disability', 'Ill', 'In retirement', 'NaN', 'School',
       'Something else/other', 'Taking care of h or f'],
      dtype='object', name='PENLFACT')

In [24]:
table2.columns

Index(['Asian_only', 'Black_only', 'Other', 'White_only'], dtype='object', name='PTDTRACE')

In [25]:
table2 = table2.reindex(index=['School','Taking care of h or f','Something else/other','Disability', 'Ill', 'In retirement',
                               ], columns=['Asian_only', 'White_only', 'Black_only', 'Other'])

table2

PTDTRACE,Asian_only,White_only,Black_only,Other
PENLFACT,,,,
School,285,2984,591,189
Taking care of h or f,258,2389,320,98
Something else/other,27,295,111,19
Disability,7,100,39,5
Ill,6,72,14,5
In retirement,5,25,4,3


In [26]:
table2.columns

Index(['Asian_only', 'White_only', 'Black_only', 'Other'], dtype='object', name='PTDTRACE')

In [27]:
total2 = 0
for Black_only in table2.columns:
    total2+= np.sum(table2[Black_only])
print(total2)

7851


In [28]:
chi2, p, dof, expected = stats.chi2_contingency(table2)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)

chi2: 114.16917702814342
p: 2.5419253641052523e-17
dof: 15
expected: [[3.03249522e+02 3.02475927e+03 5.56473188e+02 1.64518023e+02]
 [2.29552923e+02 2.28967329e+03 4.21237422e+02 1.24536365e+02]
 [3.38525029e+01 3.37661444e+02 6.21204942e+01 1.83655585e+01]
 [1.13091326e+01 1.12802828e+02 2.07526430e+01 6.13539676e+00]
 [7.26480703e+00 7.24627436e+01 1.33311680e+01 3.94128137e+00]
 [2.77111196e+00 2.76404280e+01 5.08508470e+00 1.50337537e+00]]


Statistical analysis from chi-squared test from independence indicated that there was an association between race and status (p < 0.001). chi^2(25, N = 7851)=119.822; p < 0.001

In [29]:
table2['Total'] = table2['White_only'] + table2['Other'] +table2['Asian_only'] + table2['Black_only']
table2['Black__only_per'] = table2['Black_only'] / table2['Total']
table2

PTDTRACE,Asian_only,White_only,Black_only,Other,Total,Black__only_per
PENLFACT,,,,,,
School,285,2984,591,189,4049,0.145962
Taking care of h or f,258,2389,320,98,3065,0.104405
Something else/other,27,295,111,19,452,0.245575
Disability,7,100,39,5,151,0.258278
Ill,6,72,14,5,97,0.144330
In retirement,5,25,4,3,37,0.108108


In [30]:
def CI_prop_diff(s1, s2, n1, n2):
    p1 = s1 / n1
    p2 = s2 / n2
    se2p1 = p1 * (1 - p1) / n1
    se2p2 = p2 * (1 - p2) / n2
    se2 = se2p1 + se2p2
    se = np.sqrt(se2)
    low = (p1 - p2) - 1.96 * se
    up = (p1 - p2) + 1.96 * se
    print("Proportion difference is:", p1 - p2)
    print("95% CI is: ({}, {})".format(low, up))

In [31]:
CI_prop_diff(s1=320, s2=2745, n1=3065, n2=4786)

Proportion difference is: -0.4691432801898414
95% CI is: (-0.486849854872326, -0.4514367055073568)


My hypothesis is that there is an association between race and status, concretely, people that marked down 'Black only' as race and taking care of house or children. The data provide a strong evidence that  Black people have a smaller possibility of taking care of the house or family than the probability of other races doing so. chi^2(15, N = 7851)=114.17; p < 0.001. The observed proportion difference was -0.4869(95% CI = [-0.487,-0.451]).

In [32]:
age_df3 = df3['PRTAGE'].dropna()

In [33]:
age_df3

0        85
1        25
2         3
3        27
4        50
         ..
64280    10
64281    36
64282    14
64283     3
64284    35
Name: PRTAGE, Length: 64285, dtype: int64

In [34]:
income= df3['HEFAMINC'].dropna()

In [35]:
from scipy import stats
corr = stats.pearsonr(df3['PRTAGE'], df3['HEFAMINC'])
print('Correlation coefficient:', corr[0])
print('p-value:', corr[1])

Correlation coefficient: -0.0710570653054613
p-value: 9.695675491466678e-73


In [36]:
x_vals = df3['PRTAGE'].values
y_vals = df3['HEFAMINC']

reg_model_1 = OLS(y_vals, sm.add_constant(x_vals)).fit()
display(reg_model_1.summary())


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               HEFAMINC   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     326.2
Date:                Wed, 23 Dec 2020   Prob (F-statistic):           9.70e-73
Time:                        17:31:33   Log-Likelihood:            -1.7817e+05
No. Observations:               64285   AIC:                         3.563e+05
Df Residuals:                   64283   BIC:                         3.564e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.0710      0.029    420.684      0.000      12.015      12.127
x1            -0.0120      0.001    -18.062      0.000      -0.013      -0.011
==============================================================================
Omnibus:                     6952.476   Durbin-Watson:                   0.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9499.512
Skew:                          -0.940   Prob(JB):                         0.00
Kurtosis:                       3.106   Cond. No.                         81.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Even though the p-value is really small (p < 0.001), this regression indicates that there is only a weak negative relationship of -0.071 between income and age.

In [37]:
print("Max:", max (age_df3))    
print("Min:", min (age_df3))

Max: 85
Min: 0


In [38]:
print("Max:", max (income))    
print("Min:", min (income))

Max: 16
Min: 1


In [39]:
def CI_mean(income):
    se = np.std(income)/np.sqrt(len(income))
    low = np.mean(income)-1.96*se
    up = np.mean(income)+1.96*se
    print("Average is:", np.mean(income))
    print("95% CI of mean is: ({}, {})".format(low,up))  

In [40]:
CI_mean(df3['HEFAMINC'])

Average is: 11.632029244769386
95% CI of mean is: (11.60205529991746, 11.662003189621313)


The mean of the income of the survey is 11.632, which means it is between $40,000 - $59,999 since the level 11 is $40,000 to $49,999 and level 12 is $50,000 to $59,999. 
The 95% confidence interval of the mean is between levels 11.602 and 11.662)

In [41]:
def CI_mean(age):
    se = np.std(age)/np.sqrt(len(age))
    low = np.mean(age)-1.96*se
    up = np.mean(age)+1.96*se
    print("Average is:", np.mean(age))
    print("95% CI of mean is: ({}, {})".format(low,up))

In [42]:
CI_mean(df3['PRTAGE'])

Average is: 36.5520883565373
95% CI of mean is: (36.37473032355684, 36.729446389517754)


The mean of the age of the survey is 36.55 years old. The 95% confidence interval of the mean is between ages 36.375  and 36.73)

In [43]:
def function4(row):
    if row['HEFAMINC'] ==1:
        return 'Less than $5,000'
    elif row['HEFAMINC'] == 2 or row['HEFAMINC'] == 3 or row['HEFAMINC'] == 4 or row['HEFAMINC'] ==  5:
        return '$5k to $15k'
    elif row['HEFAMINC'] == 6 or row['HEFAMINC'] == 7 or row['HEFAMINC'] == 8 or row['HEFAMINC'] == 9:
        return '$15k to $35k'
    elif row['HEFAMINC'] == 10 or row['HEFAMINC'] == 11 or row['HEFAMINC'] == 12:
        return '$35k to $60k'
    elif row['HEFAMINC'] == 13:
        return '$60K to $75k'
    elif row['HEFAMINC'] == 14:
        return '$75K to $100k'
    elif row['HEFAMINC'] == 15 or row['HEFAMINC'] == 16:
        return '$100k and over'
    else:
        return 'NaN'

In [44]:
df3['HEFAMINC'] = df3.apply(function4, axis=1)

In [45]:
table4 = pd.crosstab(index=df3["HEFAMINC"], columns=df3["PRTAGE"])

In [46]:
table4

PRTAGE,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,85
HEFAMINC,,,,,,,,,,,,,,,,,,,,,
$100k and over,207,257,264,289,293,311,303,337,311,322,...,88,66,59,64,41,40,38,28,118,100
$15k to $35k,113,143,152,163,163,169,160,174,199,183,...,125,114,106,86,101,88,89,82,316,338
$35k to $60k,135,167,168,162,187,202,200,192,233,231,...,114,89,90,99,98,71,72,90,267,267
$5k to $15k,46,62,57,53,73,65,61,58,56,66,...,62,49,45,42,32,34,41,29,158,168
$60K to $75k,66,81,99,85,116,113,102,112,93,100,...,56,41,37,44,39,32,21,19,92,71
$75K to $100k,85,114,117,130,125,119,137,123,137,126,...,55,40,34,31,30,32,20,24,79,78
"Less than $5,000",8,27,30,20,26,30,23,21,19,18,...,7,7,2,4,5,4,5,8,10,14


In [47]:
table4 = table4.reindex(index=['Less than $5,000','$5k to $15k', '$15k to $35k', '$35k to $60k', '$60K to $75k', '$75K to $100k',  '$100k and over'])
table4

PRTAGE,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,85
HEFAMINC,,,,,,,,,,,,,,,,,,,,,
"Less than $5,000",8,27,30,20,26,30,23,21,19,18,...,7,7,2,4,5,4,5,8,10,14
$5k to $15k,46,62,57,53,73,65,61,58,56,66,...,62,49,45,42,32,34,41,29,158,168
$15k to $35k,113,143,152,163,163,169,160,174,199,183,...,125,114,106,86,101,88,89,82,316,338
$35k to $60k,135,167,168,162,187,202,200,192,233,231,...,114,89,90,99,98,71,72,90,267,267
$60K to $75k,66,81,99,85,116,113,102,112,93,100,...,56,41,37,44,39,32,21,19,92,71
$75K to $100k,85,114,117,130,125,119,137,123,137,126,...,55,40,34,31,30,32,20,24,79,78
$100k and over,207,257,264,289,293,311,303,337,311,322,...,88,66,59,64,41,40,38,28,118,100


In [60]:
from scipy import stats

f, p = stats.f_oneway(
    table4.iloc[0].values,
    table4.iloc[1].values,
    table4.iloc[2].values,
    table4.iloc[3].values,
    table4.iloc[4].values,
    table4.iloc[5].values,
    table4.iloc[6].values)
print("F:", f)
print("p:", p)

F: 197.28383218795304
p: 2.88563958785491e-135


There was a statistically significant difference between groups as determined by one-way ANOVA (F = 197.284, p <0.01). 

In [61]:
maritalstatus= df3['PEMARITL'].dropna()

In [62]:
maritalstatus

0        3
1        6
2       -1
3        6
4        2
        ..
64280   -1
64281    4
64282   -1
64283   -1
64284    6
Name: PEMARITL, Length: 64285, dtype: int64

In [63]:
def function5(row):
    if row['PEMARITL'] == 1:
        return 'Married_spousep'
    elif row['PEMARITL'] == 2:
        return 'Married_spousea'
    elif row['PEMARITL'] == 3:
        return 'Widowed'
    elif row['PEMARITL'] == 4:
        return 'Divorced'
    elif row['PEMARITL'] == 5:
        return 'Separated'
    elif row['PEMARITL'] == 6:
        return 'Never_married'
    else:
        return 'NaN'

In [64]:
df3['PEMARITL'] = df3.apply(function5, axis=1)

In [65]:
table5  = pd.crosstab(index=df3["PENLFACT"], columns=df3["PEMARITL"])

In [66]:
table5

PEMARITL,Divorced,Married_spousea,Married_spousep,NaN,Never_married,Separated,Widowed
PENLFACT,,,,,,,
Disability,29,2,43,0,52,10,15
Ill,15,1,37,0,38,5,1
In retirement,6,0,19,0,7,1,4
NaN,4618,666,22319,14823,10368,919,2721
School,37,55,120,0,3799,32,6
Something else/other,36,5,124,0,268,7,12
Taking care of h or f,117,34,2222,0,602,56,34


In [67]:
table5 = table5.reindex(index=['Taking care of h or f','Disability', 'Ill', 'In retirement', 'School',
                          'Something else/other'],columns=['Married_spousep','Never_married'
                                                           ,'Divorced','Separated','Widowed','Married_spousea'])
table5

PEMARITL,Married_spousep,Never_married,Divorced,Separated,Widowed,Married_spousea
PENLFACT,,,,,,
Taking care of h or f,2222,602,117,56,34,34
Disability,43,52,29,10,15,2
Ill,37,38,15,5,1,1
In retirement,19,7,6,1,4,0
School,120,3799,37,32,6,55
Something else/other,124,268,36,7,12,5


In [68]:
table5.columns

Index(['Married_spousep', 'Never_married', 'Divorced', 'Separated', 'Widowed',
       'Married_spousea'],
      dtype='object', name='PEMARITL')

In [69]:
total5 = 0
for Married_spousep in table5.columns:
    total5 += np.sum(table5[Married_spousep])
print(total5)

7851


In [70]:
chi2, p, dof, expected = stats.chi2_contingency(table5)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)

chi2: 4781.312458197405
p: 0.0
dof: 25
expected: [[1.00136607e+03 1.86062795e+03 9.36950707e+01 4.33339702e+01
  2.81085212e+01 3.78684244e+01]
 [4.93332060e+01 9.16655203e+01 4.61597249e+00 2.13488728e+00
  1.38479175e+00 1.86562221e+00]
 [3.16908674e+01 5.88844733e+01 2.96522736e+00 1.37141765e+00
  8.89568208e-01 1.19844606e+00]
 [1.20882690e+01 2.24610878e+01 1.13106611e+00 5.23118074e-01
  3.39319832e-01 4.57139218e-01]
 [1.32284868e+03 2.45797147e+03 1.23775315e+02 5.72460833e+01
  3.71325946e+01 5.00258566e+01]
 [1.47672908e+02 2.74389505e+02 1.38173481e+01 6.39052350e+00
  4.14520443e+00 5.58451153e+00]]


In [71]:
table5['Total'] = table5['Married_spousep'] + table5['Never_married'] +table5['Divorced'] +table5['Separated'] +table5['Widowed']+table5['Married_spousea']
table5['Married_spousep_per'] = table5['Married_spousep'] / table5['Total']
table5

PEMARITL,Married_spousep,Never_married,Divorced,Separated,Widowed,Married_spousea,Total,Married_spousep_per
PENLFACT,,,,,,,,
Taking care of h or f,2222,602,117,56,34,34,3065,0.724959
Disability,43,52,29,10,15,2,151,0.284768
Ill,37,38,15,5,1,1,97,0.381443
In retirement,19,7,6,1,4,0,37,0.513514
School,120,3799,37,32,6,55,4049,0.029637
Something else/other,124,268,36,7,12,5,452,0.274336


In [72]:
def CI_prop_diff(s1, s2, n1, n2):
    p1 = s1 / n1
    p2 = s2 / n2
    se2p1 = p1 * (1 - p1) / n1
    se2p2 = p2 * (1 - p2) / n2
    se2 = se2p1 + se2p2
    se = np.sqrt(se2)
    low = (p1 - p2) - 1.96 * se
    up = (p1 - p2) + 1.96 * se
    print("Proportion difference is:", p1 - p2)
    print("95% CI is: ({}, {})".format(low, up))

In [73]:
CI_prop_diff(s1=2222, s2=843, n1=3065, n2=4786)

Proportion difference is: 0.5488204789799505
95% CI is: (0.5296790522942718, 0.5679619056656291)


My hypothesis is that there is an association between marital status and status, concretely, people that are married with their spouse present and taking care of house or children. The data provide a strong evidence that the probability of married people with their spouse home have a bigger possibility of taking care of the house or family than the probability of people with other marrital status of doing so. chi^2(25, N = 7851)=4781.312; p < 0.001. The observed proportion difference was 0.549 (95% CI = [0.529,0.567]).